In [25]:
import cv2
import tkinter as tk
from tkinter.simpledialog import askstring
from tkinter import filedialog
from PIL import Image, ImageTk
import os

class WebcamApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)

        self.video_capture = cv2.VideoCapture(0)
        self.current_frame = None
        self.image_path = ""

        self.video_frame = tk.Label(self.window)
        self.video_frame.grid(row=0, column=0, columnspan=2)

        self.capture_button = tk.Button(self.window, text="Capture", width=10, command=self.capture_image)
        self.capture_button.grid(row=1, column=0, padx=10, pady=5)

        self.browse_button = tk.Button(self.window, text="Browse", width=10, command=self.browse_directory)
        self.browse_button.grid(row=1, column=1, padx=10, pady=5)

        self.exit_button = tk.Button(self.window, text="Exit", width=10, command=self.exit_app)
        self.exit_button.grid(row=2, column=0, columnspan=2, pady=5)

        self.update()
        self.window.protocol("WM_DELETE_WINDOW", self.exit_app)  # Bind close button to exit_app
        self.window.mainloop()

    def update(self):
        ret, frame = self.video_capture.read()
        if ret:
            self.current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(self.current_frame))
            self.video_frame.config(image=self.photo)
        self.window.after(10, self.update)

    def capture_image(self):
        if self.current_frame is not None and self.image_path:
            image_name = askstring("Image Name", "Enter a name for the image:")
            if image_name:
                image_filename = os.path.join(self.image_path, f"{image_name}.jpg")
                resized_frame = cv2.resize(self.current_frame, (640, 480))
                cv2.imwrite(image_filename, cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR))
                print(f"Photo captured and saved as {image_filename}")

    def browse_directory(self):
        self.image_path = filedialog.askdirectory()
        print(f"Image storage directory set to: {self.image_path}")

    def exit_app(self):
        self.video_capture.release()
        self.window.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = WebcamApp(root, "Webcam Capture App")


Image storage directory set to: C:/Users/vaibhav_cs/OneDrive/Desktop/New folder/project(ML)
Photo captured and saved as C:/Users/vaibhav_cs/OneDrive/Desktop/New folder/project(ML)\vaibahv.jpg


In [1]:
import os
import csv
import cv2
import numpy as np
import h5py
from tkinter import filedialog, messagebox
import tkinter as tk
from PIL import Image, ImageTk

class ImageLabelingApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.images_folder = ""
        self.images_list = []
        self.current_image_index = 0
        self.labels = {}
        self.image_arrays = []

        self.label_frame = tk.LabelFrame(self.window, text="Labeling")
        self.label_frame.pack(padx=20, pady=20)

        self.label_var = tk.StringVar()
        self.label_var.set("")

        self.image_label = tk.Label(self.label_frame)
        self.image_label.pack()

        self.sleeping_button = tk.Button(self.label_frame, text="Sleeping", command=lambda: self.label_image("sleeping"))
        self.sleeping_button.pack(side=tk.LEFT, padx=10)

        self.awake_button = tk.Button(self.label_frame, text="Awake", command=lambda: self.label_image("awake"))
        self.awake_button.pack(side=tk.LEFT, padx=10)

        self.skip_button = tk.Button(self.label_frame, text="Skip", command=self.skip_image)
        self.skip_button.pack(side=tk.LEFT, padx=10)

        self.load_images()
        self.load_labels()

        self.window.mainloop()

    def load_images(self):
        self.images_folder = filedialog.askdirectory(title="Select Images Folder")
        if self.images_folder:
            self.images_list = [filename for filename in os.listdir(self.images_folder) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
            self.show_image()

    def load_labels(self):
        csv_file = os.path.join(self.images_folder, "labels.csv")
        if os.path.exists(csv_file):
            with open(csv_file, "r") as file:
                reader = csv.reader(file)
                self.labels = {row[0]: row[1] for row in reader}

    def show_image(self):
        if self.images_list and 0 <= self.current_image_index < len(self.images_list):
            image_path = os.path.join(self.images_folder, self.images_list[self.current_image_index])
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.image_arrays.append(image)
            image = cv2.resize(image, (400, 300))
            image = Image.fromarray(image)
            photo = ImageTk.PhotoImage(image=image)
            self.image_label.config(image=photo)
            self.image_label.image = photo

    def label_image(self, label):
        if self.images_list and 0 <= self.current_image_index < len(self.images_list):
            filename = self.images_list[self.current_image_index]
            self.labels[filename] = label
            self.current_image_index += 1
            self.show_image()
            self.save_labels()

    def skip_image(self):
        if self.images_list:
            self.current_image_index += 1
            self.show_image()

    def save_labels(self):
        csv_file = os.path.join(self.images_folder, "labels.csv")
        with open(csv_file, "w", newline="") as file:
            writer = csv.writer(file)
            for filename, label in self.labels.items():
                writer.writerow([filename, label])

        if self.current_image_index >= len(self.images_list):
            self.convert_to_grayscale_and_flatten()

    def convert_to_grayscale_and_flatten(self):
        grayscale_data = []

        for filename, label in self.labels.items():
            image_path = os.path.join(self.images_folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            flattened_image = image.flatten()
            
            # Convert label to a numeric value
            label_numeric = 1 if label == "sleeping" else 0
            
            # Append label_numeric to the flattened_image array
            flattened_image = np.append(flattened_image, label_numeric)
            
            grayscale_data.append(flattened_image)

        grayscale_hdf5_file = os.path.join(self.images_folder, "grayscale_data.h5")
        with h5py.File(grayscale_hdf5_file, 'w') as hdf5_file:
            hdf5_file.create_dataset('data', data=grayscale_data)

        messagebox.showinfo("Conversion Done", "Grayscale and flattening conversion is complete!")
        self.window.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageLabelingApp(root, "Image Labeling and Data Conversion App")


In [28]:
import h5py
import pandas as pd

# Load the HDF5 file
hdf5_file_path = "grayscale_data.h5"  # Update with the actual path
hdf5_file = h5py.File(hdf5_file_path, 'r')

# Load the data from the HDF5 file
data = hdf5_file['data'][:]

# Close the HDF5 file
hdf5_file.close()

# Convert the data into a pandas DataFrame
# Assuming the last column contains labels
df_columns = [f'pixel_{i}' for i in range(data.shape[1] - 1)] + ['label']
df = pd.DataFrame(data, columns=df_columns)

# Show the DataFrame
print(df)


   pixel_0  pixel_1  pixel_2  pixel_3  pixel_4  pixel_5  pixel_6  pixel_7  \
0        4        4        4        3        2        2        2        2   
1        6        6        5        5        5        4        4        5   
2      126      119      114      117      119      115      114      118   

   pixel_8  pixel_9  ...  pixel_307191  pixel_307192  pixel_307193  \
0        3        4  ...            29            28            28   
1        6        4  ...            23            24            25   
2      130      129  ...            47            46            48   

   pixel_307194  pixel_307195  pixel_307196  pixel_307197  pixel_307198  \
0            27            25            24            22            22   
1            27            28            28            29            31   
2            53            46            43            47            44   

   pixel_307199  label  
0            22      1  
1            31      1  
2            46      0  

[3 rows 

In [29]:
df.shape

(3, 307201)

In [30]:
X = df.iloc[:, :-1].values
y = df['label'].values

# Show X and y
print("X:")
print(X)
print("\ny:")
print(y)

X:
[[  4   4   4 ...  22  22  22]
 [  6   6   5 ...  29  31  31]
 [126 119 114 ...  47  44  46]]

y:
[1 1 0]


In [31]:
X=X/255
X

array([[0.01568627, 0.01568627, 0.01568627, ..., 0.08627451, 0.08627451,
        0.08627451],
       [0.02352941, 0.02352941, 0.01960784, ..., 0.11372549, 0.12156863,
        0.12156863],
       [0.49411765, 0.46666667, 0.44705882, ..., 0.18431373, 0.17254902,
        0.18039216]])